In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

In [3]:
# Define file names
file_names = [
    "10Y_DGS10.csv",
    "CPI.csv",
    "FEDFUNDS.csv",
    "GDP.csv",
    "INF_Sticky.csv",
    "M2.csv",
    "UNRATE.csv"
]

# Initialize an empty list to store DataFrames
dfs = []

# Loop through each CSV file
for file in file_names:
    # Read the CSV file
    df = pd.read_csv(file, parse_dates=['observation_date'])
    
    # Set 'observation_date' as the index
    df.set_index('observation_date', inplace=True)
    
    # Append DataFrame to the list
    dfs.append(df)

# Merge all DataFrames on the 'observation_date' index
merged_df = dfs[0]
for df in dfs[1:]:
    merged_df = merged_df.merge(df, how='outer', left_index=True, right_index=True)

# Check the first few rows of the merged DataFrame
print(merged_df.head())

                  10Y Treasury Yield  CPIAUCSL  FEDFUNDS      GDP  \
observation_date                                                    
1947-01-01                       NaN     21.48       NaN  243.164   
1947-02-01                       NaN     21.62       NaN      NaN   
1947-03-01                       NaN     22.00       NaN      NaN   
1947-04-01                       NaN     22.00       NaN  245.968   
1947-05-01                       NaN     21.95       NaN      NaN   

                  CORESTICKM159SFRBATL  WM2NS  UNRATE  
observation_date                                       
1947-01-01                         NaN    NaN     NaN  
1947-02-01                         NaN    NaN     NaN  
1947-03-01                         NaN    NaN     NaN  
1947-04-01                         NaN    NaN     NaN  
1947-05-01                         NaN    NaN     NaN  


In [4]:
# Find all null values in merged_df
null_values = merged_df.isnull().sum()

# Print the columns with null values and their counts
print("Null values in each column:")
print(null_values[null_values > 0])

Null values in each column:
10Y Treasury Yield       1107
CPIAUCSL                16038
FEDFUNDS                16128
GDP                     16667
CORESTICKM159SFRBATL    16290
WM2NS                   14649
UNRATE                  16050
dtype: int64


In [5]:
# Forward fill the null values
merged_df.ffill(inplace=True)

# Optional: Verify by checking if there are any more null values
null_values_after_ffill = merged_df.isnull().sum()

print("Null values after forward filling:")
print(null_values_after_ffill[null_values_after_ffill > 0])

Null values after forward filling:
10Y Treasury Yield       181
FEDFUNDS                  90
CORESTICKM159SFRBATL    1765
WM2NS                   5160
UNRATE                    12
dtype: int64


In [6]:
# Ensure the index is datetime
merged_df.index = pd.to_datetime(merged_df.index)

# Resample CPI to monthly frequency (use the last CPI value of each month)
monthly_cpi = merged_df['CPIAUCSL'].resample('M').last()

# Calculate YoY inflation and round to 2 decimal places
monthly_inflation = ((monthly_cpi - monthly_cpi.shift(12)) / monthly_cpi.shift(12) * 100).round(2)

# Reindex back to daily and forward-fill to assign inflation to each day
daily_inflation = monthly_inflation.reindex(merged_df.index, method='ffill')

# Assign to the DataFrame
merged_df['Inflation'] = daily_inflation

C:\Users\steve\AppData\Local\Temp\ipykernel_12740\3047265482.py:5: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_cpi = merged_df['CPIAUCSL'].resample('M').last()


In [7]:
# Check the first few rows of the merged DataFrame
print(merged_df.head())

                  10Y Treasury Yield  CPIAUCSL  FEDFUNDS      GDP  \
observation_date                                                    
1947-01-01                       NaN     21.48       NaN  243.164   
1947-02-01                       NaN     21.62       NaN  243.164   
1947-03-01                       NaN     22.00       NaN  243.164   
1947-04-01                       NaN     22.00       NaN  245.968   
1947-05-01                       NaN     21.95       NaN  245.968   

                  CORESTICKM159SFRBATL  WM2NS  UNRATE  Inflation  
observation_date                                                  
1947-01-01                         NaN    NaN     NaN        NaN  
1947-02-01                         NaN    NaN     NaN        NaN  
1947-03-01                         NaN    NaN     NaN        NaN  
1947-04-01                         NaN    NaN     NaN        NaN  
1947-05-01                         NaN    NaN     NaN        NaN  


In [8]:
# First, rename the columns (without resetting the index)
merged_df.columns = [
    '10Y_Treasury_Yield',     # Column 1 (after index)
    'CPI',                    # Column 2
    'Fed_Funds_Rate',              # Column 3
    'GDP',                    # Column 4
    'Inflation_Sticky',       # Column 5
    'M2_Money_Supply',        # Column 6
    'Unemployment Rate',      # Column 7
    'Inflation'               # Column 8
]

# Then rename the index itself to 'Date'
merged_df.index.name = 'Date'

In [9]:
# Filter rows where 'observation_date' is on or after January 1, 1990
merged_df = merged_df[merged_df.index.year >= 1981]

# Optional: Confirm the change by printing the first few rows
print("Data after 1990:")
print(merged_df.head())

Data after 1990:
            10Y_Treasury_Yield   CPI  Fed_Funds_Rate       GDP  \
Date                                                             
1981-01-01               12.43  87.2           19.08  3124.206   
1981-01-02               12.42  87.2           19.08  3124.206   
1981-01-05               12.15  87.2           19.08  3124.206   
1981-01-06               12.11  87.2           19.08  3124.206   
1981-01-07               12.38  87.2           19.08  3124.206   

            Inflation_Sticky  M2_Money_Supply  Unemployment Rate  Inflation  
Date                                                                         
1981-01-01         12.018101           1601.8                7.5      12.35  
1981-01-02         12.018101           1601.8                7.5      12.35  
1981-01-05         12.018101           1620.7                7.5      12.35  
1981-01-06         12.018101           1620.7                7.5      12.35  
1981-01-07         12.018101           1620.7       

In [10]:
# Ensure the index is datetime
merged_df.index = pd.to_datetime(merged_df.index)

# Create a full daily date range from min to max date
full_index = pd.date_range(start=merged_df.index.min(), end=merged_df.index.max(), freq='D')

# Reindex the DataFrame to include all daily dates
merged_df = merged_df.reindex(full_index)

# Forward-fill missing values to populate new rows with the last known data
merged_df = merged_df.ffill()

# Optional: Rename the index to 'Date' for clarity
merged_df.index.name = 'Date'

In [11]:
# Save the merged DataFrame to a CSV file
merged_df.to_csv("macro_data_1981.csv")

In [15]:
# Load the daily macro data
merged_df = pd.read_csv("macro_data_1981.csv", parse_dates=['Date'], index_col='Date')

# Truncate the data at June 2025
cutoff_date = "2025-06-30"
merged_df = merged_df.loc[:cutoff_date]

# Resample to monthly frequency using the last observation of each month
monthly_df = merged_df.resample('M').last()

# Save to CSV
monthly_df.to_csv("macro_data_monthly.csv")

# Preview
print(monthly_df.tail())

            10Y_Treasury_Yield      CPI  Fed_Funds_Rate        GDP  \
Date                                                                 
2025-02-28                4.24  319.775            4.33  29962.047   
2025-03-31                4.23  319.615            4.33  29962.047   
2025-04-30                4.17  320.321            4.33  29962.047   
2025-05-31                4.41  320.580            4.33  29962.047   
2025-06-30                4.24  321.500            4.33  29962.047   

            Inflation_Sticky  M2_Money_Supply  Unemployment Rate  Inflation  
Date                                                                         
2025-02-28          3.516515          21517.7                4.1       2.81  
2025-03-31          3.257815          21912.7                4.2       2.41  
2025-04-30          3.176690          21720.1                4.2       2.33  
2025-05-31          3.159131          21738.7                4.2       2.33  
2025-06-30          3.288774          220

C:\Users\steve\AppData\Local\Temp\ipykernel_12740\3209997627.py:9: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  monthly_df = merged_df.resample('M').last()
